In [101]:
import vcf
import pandas
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import numpy
from bokeh.models import HoverTool, ColumnDataSource
from bokeh.plotting import figure
%load_ext autoreload
%autoreload 2
from bokeh.io import output_notebook, show
output_notebook()
import matplotlib
from bokeh.io import output_file
%matplotlib inline

Loading BokehJS ...

## Load 1000 Genomes Data

In [ ]:
# VCF file was taken from here:
# ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/ALL.chr20.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz

snps = {}

with open("ALL.chr20.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf",'r') as handle:
    
    line = handle.readline()
    line = line.rstrip("\n")
    while line.startswith("##"):
        line = handle.readline()
    
    line=line.rstrip("\n")
    header = line.split("\t")
    sample_names = header[9:]
    for sample_name in sample_names:
        snps[sample_name] = {}
        
    line_count = 0
        
    for line in handle:
        line=line.rstrip("\n")
        fields = line.split("\t")
        site_info = {}
        # Was getting unreadable as a dict comp
        for field in fields[7].split(";"):
            split_field = field.split("=")
            if len(split_field) ==2:
                site_info[split_field[0]] = split_field[1]
        # Skip indels
        if len(fields[3]) == 1 and len(fields[4]) == 1 and float(site_info["AF"]) > 0.05:
            
            # Set sample columns to allele counts
            for genotype, sample_name in zip(fields[9:],sample_names):
                # Trying to filter out complex genotypes
                if genotype not in ["0|0","1|0","0|1","1|1"]:
                    allele_count = numpy.nan
                else:
                    allele_count = genotype.count("1")
                snps[sample_name][fields[0]+"_"+fields[1]] = allele_count
            line_count += 1

### Load population map

In [78]:
population_map = {}
population_metadata_map = {}
with open("20131219.populations.tsv",'r') as handle:
    handle.readline()
    for line in handle:
        fields = line.split("\t")
        population_metadata_map[fields[1]] = {"superpopulation":fields[2],"fullname":fields[0]}
with open("integrated_call_samples_v2.20130502.ALL.ped",'r') as handle:
    handle.readline()
    for line in handle:
        fields = line.split("\t")
        if fields[6] in population_metadata_map:
            population_map[fields[0]] = population_metadata_map[fields[6]]
        else:
            population_map[fields[0]] = {"superpopulation":"Unknown","fullname":"Unknown"}

In [41]:
len(snps[sample_name])

501

In [87]:
snp_data_frame = pandas.DataFrame(snps)
snp_data_frame = snp_data_frame.dropna()
snp_data_frame = snp_data_frame.transpose()

In [70]:
snp_data_frame

,20_100272,20_100495,20_100505,20_100699,20_101362,20_102181,20_102799,20_103517,20_103642,20_104520,...,20_94528,20_94592,20_94952,20_95093,20_96931,20_97122,20_97394,20_97395,20_98930,20_99801
HG00096,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
HG00097,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
HG00099,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
HG00100,0,0,0,1,1,1,1,1,0,0,...,0,0,1,0,1,1,1,0,1,0
HG00101,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
HG00102,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
HG00103,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
HG00105,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
HG00106,1,1,1,0,0,1,0,1,1,1,...,1,1,0,1,1,0,1,1,0,1
HG00107,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Run PCA

In [88]:
# mean center and unit variance
snp_data_frame[snp_data_frame.columns] = StandardScaler().fit_transform(snp_data_frame)

In [89]:
pca = PCA(n_components=3)
snp_transformed_values = pca.fit_transform(snp_data_frame)

In [75]:
print()

### Plot PCA Results

In [100]:
color_map = {"AFR":'red',"AMR":"green","EAS":"yellow","EUR":"blue","SAS":"purple"}
sample_dict = {"sample_id":[],"sample_color":[],"population_name":[],"PC1":snp_transformed_values.T[0],"PC2":snp_transformed_values.T[1],"PC3":snp_transformed_values.T[2]}
for sample_id in snp_data_frame.index:
    sample_dict["sample_id"].append(sample_id)
    if sample_id in population_map:
        sample_color = color_map[population_map[sample_id]["superpopulation"]]
        population = population_map[sample_id]["fullname"]
    else:
        sample_color = "black"
        population = "Unknown"
    sample_dict["population_name"].append(population)
    sample_dict["sample_color"].append(sample_color)

plot_df = pandas.DataFrame(sample_dict) 
plot_df.head(10)

,PC1,PC2,PC3,population_name,sample_color,sample_id
0,-6.469806,4.596127,-7.632224,British in England and Scotland,blue,HG00096
1,-5.614342,6.027925,-4.861604,British in England and Scotland,blue,HG00097
2,-8.275310,3.702969,-8.287270,British in England and Scotland,blue,HG00099
3,13.215349,13.475030,0.330324,British in England and Scotland,blue,HG00100
4,-4.202166,7.131212,-3.221282,British in England and Scotland,blue,HG00101
5,-6.499189,-4.819168,-11.005335,British in England and Scotland,blue,HG00102
6,-5.870688,5.739359,-4.121535,British in England and Scotland,blue,HG00103
7,2.606298,6.423772,6.792039,British in England and Scotland,blue,HG00105
8,-6.673476,-12.321377,3.237398,British in England and Scotland,blue,HG00106
9,-21.818971,-3.865494,7.185898,British in England and Scotland,blue,HG00107


In [102]:
plot_handle = figure()
hover = HoverTool(tooltips=[("Sample ID: ", "@sample_id"),("Population Name: ","@population_name")])
plot_handle.add_tools(hover)
data_source = ColumnDataSource(plot_df)
function = getattr(plot_handle,"scatter")
plot_handle.scatter("PC1", "PC2", source=data_source,color=plot_df["sample_color"])
show(plot_handle)

/Users/mschultz/anaconda/lib/python3.5/site-packages/bokeh/util/deprecation.py:33: BokehDeprecationWarning: 
Supplying a user-defined data source AND iterable values to glyph methods is deprecated.

See https://github.com/bokeh/bokeh/issues/2056 for more information.

  warn(message)
/Users/mschultz/anaconda/lib/python3.5/site-packages/bokeh/util/deprecation.py:33: BokehDeprecationWarning: 
Supplying a user-defined data source AND iterable values to glyph methods is deprecated.

See https://github.com/bokeh/bokeh/issues/2056 for more information.

  warn(message)


## Run Autoencoder